# Chattanooga 311 EDA

In [1]:
import pandas as pd
import re

In [2]:
dat = pd.read_csv(r'data/311_Service_Requests.csv', encoding='Latin-1', low_memory=False)
dat.sample(10)

,City,Created Date,Description,Intake Form,Request Type Code,Service Request Key,Service Request Number,State,Status Code,Status Date,...,X Coordinate,Y Coordinate,ZIP Code,Block Level Latitude,Block Level Longitude,Block Level Location,Address Latitude,Address Longitude,Address Location,Count
1604720,NaN,01/05/2017 03:57:20 PM,both sides of pipe under driveway is clogged a...,gov.publicstuff.com,URGENT,2228102,NaN,NaN,O-CLOSED,NaN,...,-8.522870e+01,35.032330,NaN,NaN,NaN,NaN,NaN,NaN,"(35.0323296074633, -85.2287009067461)",NaN
1405176,CHATTANOOGA,03/19/2003 03:16:49 PM,TO - Traffic Signal Bulb Out,PHONE,STANDARD,1242552,03-00006918,TN,C-CLOSED,03/21/2003 11:46:00 AM,...,NaN,NaN,37411,NaN,NaN,NaN,35.016680,-85.226156,NaN,NaN
1342546,CHATTANOOGA,10/23/2013 12:05:45 PM,BLD - Heat/Air Problems,INTERNAL,STANDARD,136761479,13-00126013,TN,C-CLOSED,10/23/2013 02:46:32 PM,...,2.175579e+06,258303.582847,37402,35.040920,-85.308266,"(35.0409197856189, -85.3082658111241)",35.040715,-85.308091,NaN,NaN
1674585,NaN,02/10/2018 01:16:21 AM,NaN,Iframe,URGENT,3760626,NaN,NaN,O-CLOSED,NaN,...,-8.516955e+01,35.086506,NaN,NaN,NaN,NaN,NaN,NaN,"(35.0865062896784, -85.169550427403)",NaN
1473006,CHATTANOOGA,11/29/2004 02:53:31 PM,NSC - Abandoned/Inoperable Vehicle,PHONE,STANDARD,8726477,04-00071970,TN,C-CLOSED,11/29/2004 12:00:00 AM,...,NaN,NaN,37404,NaN,NaN,NaN,35.028773,-85.277225,NaN,NaN
1588030,NaN,04/13/2017 07:22:16 PM,@Standifer Gap,gov.publicstuff.com,URGENT,2530589,NaN,NaN,O-CLOSED,NaN,...,-8.512921e+01,35.042860,NaN,NaN,NaN,NaN,NaN,NaN,"(35.0428601111568, -85.1292112070425)",NaN
1334831,CHATTANOOGA,06/01/2011 01:37:54 PM,CWS - Brush Collection,PHONE,STANDARD,96175538,11-00060108,TN,C-CLOSED,08/15/2011 12:12:37 PM,...,2.200197e+06,248729.721065,37411,35.014112,-85.226288,"(35.0141120799328, -85.2262877140673)",35.015166,-85.225659,NaN,NaN
783644,CHATTANOOGA,10/21/2014 12:38:12 PM,CWS - Brush Collection,PHONE,STANDARD,154071803,14-00138294,TN,C-CLOSED,10/21/2014 03:22:38 PM,...,2.187251e+06,297405.772910,37415,35.148109,-85.268302,"(35.1481089613272, -85.268302374388)",35.148304,-85.268923,NaN,NaN
179195,CHATTANOOGA,03/31/2016 11:12:55 AM,CWS - Trash Flash/Bulk Item,PHONE,STANDARD,177244425,16-00027040,TN,C-CLOSED,04/01/2016 09:34:53 AM,...,2.194742e+06,267492.549458,37406,35.065774,-85.244018,"(35.0657744377118, -85.2440180657747)",35.065862,-85.244123,NaN,NaN
123810,CHATTANOOGA,08/28/2012 09:25:27 AM,CWS - Brush Collection,PHONE,STANDARD,116717013,12-00088247,TN,C-CLOSED,08/29/2012 02:54:44 PM,...,2.184617e+06,245049.731263,37407,35.004329,-85.278403,"(35.0043294046759, -85.2784029128681)",35.004337,-85.278333,NaN,NaN


In [3]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1693874 entries, 0 to 1693873
Data columns (total 25 columns):
City                      object
Created Date              object
Description               object
Intake Form               object
Request Type Code         object
Service Request Key       int64
Service Request Number    object
State                     object
Status Code               object
Status Date               object
Street Direction          object
Street Name               object
Street Number             float64
Street Type               object
Type Code                 object
X Coordinate              float64
Y Coordinate              float64
ZIP Code                  object
Block Level Latitude      float64
Block Level Longitude     float64
Block Level Location      object
Address Latitude          float64
Address Longitude         float64
Address Location          object
Count                     float64
dtypes: float64(8), int64(1), object(16)
memory usage: 

In [4]:
dat.columns = [x.lower() for x in list(dat.columns)]
dat.columns = [x.replace(' ', '_') for x in list(dat.columns)]
dat.columns

Index(['city', 'created_date', 'description', 'intake_form',
       'request_type_code', 'service_request_key', 'service_request_number',
       'state', 'status_code', 'status_date', 'street_direction',
       'street_name', 'street_number', 'street_type', 'type_code',
       'x_coordinate', 'y_coordinate', 'zip_code', 'block_level_latitude',
       'block_level_longitude', 'block_level_location', 'address_latitude',
       'address_longitude', 'address_location', 'count'],
      dtype='object')

In [5]:
# Strip out records with no descriptions
dat = dat[~dat['description'].isna()]

In [6]:
# Description lengths
dat['descript_char_len'] = dat['description'].map(lambda x: len(x))
dat['descript_word_len'] = dat['description'].map(lambda x: len(x.split()))

In [7]:
dat.sample(25).loc[:, 'status_date':'zip_code']

,status_date,street_direction,street_name,street_number,street_type,type_code,x_coordinate,y_coordinate,zip_code
1464775,12/11/2008 02:13:56 PM,NaN,CHAMBERLAIN,1300.0,AVE,CWSTRASH,2.183006e+06,256992.324989,37404
657442,01/05/2004 11:00:00 AM,NaN,BURNS,3517.0,TRL,CWSTRASH,2.158608e+06,259433.357000,37419
522230,06/08/2010 08:31:19 AM,NaN,CHICKASAW,406.0,RD,CWSBRUS1,2.198479e+06,251912.940779,37411
1164103,06/30/2006 09:23:48 AM,E,10TH,900.0,ST,CWSMOWRI,NaN,NaN,37403
1567179,NaN,NaN,NaN,NaN,NaN,NaN,-8.527624e+01,35.032444,NaN
392072,06/14/2011 10:40:27 AM,NaN,FOREST,2799.0,RD,TAGENERA,2.199820e+06,274719.724878,37406
141200,06/12/2008 09:19:56 AM,NaN,ASHLAND,600.0,TER,TAGENERA,2.183943e+06,286435.370104,37415
1446591,07/21/2004 12:52:00 PM,NaN,OAKWOOD,4628.0,DR,PARWORKR,2.215525e+06,275160.433655,37416
7818,05/12/2015 03:32:08 PM,NaN,CANNONDALE,1757.0,LOOP,CWSTRASH,2.221335e+06,251125.072962,37421
1024102,08/19/2009 12:00:00 AM,S,BEECH,1801.0,ST,NSCABAND,2.183834e+06,251891.144249,37404


## Small Dataset

In [45]:
chattanooga_311_compact = dat[['description', 'request_type_code', 'type_code', 
                               'service_request_key'
                              ]]

In [47]:
chattanooga_311_compact['city'] = 'US_CHATTANOOGA'
chattanooga_311_compact['complaint_id'] = chattanooga_311_compact['city'] + '_' + chattanooga_311_compact['service_request_key'].map(str)


c:\users\alexl\envs\mlbase\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
c:\users\alexl\envs\mlbase\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [48]:
chattanooga_311_compact = chattanooga_311_compact.drop('service_request_key', axis=1)

In [50]:
chattanooga_311_compact = chattanooga_311_compact[['complaint_id', 'city', 'description', 
                                                   'request_type_code', 'type_code']]

In [51]:
chattanooga_311_compact.to_csv('output/chattanoogle_311_compact.csv', index_label='index')

In [94]:
dat['description'].value_counts().to_csv('output\description_counts.csv')

## Cities
Are there differences between records with different cities?

In [83]:
dat['city'].value_counts(dropna=False)

CHATTANOOGA                            1415024
NaN                                     179101
DUPLICATE                                 1284
Chattanooga                               1241
HIXSON                                     678
RED BANK                                   150
CHATT.                                     120
chattanooga                                111
UNINCORPORATED                              76
EAST RIDGE                                  48
Hixson                                      41
INVALID                                     41
CHATT                                       40
CHATTANOOGAH                                40
LUPTON CITY                                 39
HAMILTON COUNTY                             39
TEST                                        31
UNKNOWN                                     26
OOLTEWAH                                    22
CHATTANOOG                                  20
Chattanoooga                                15
CHATTANOOGAHA

In [84]:
print("CHATTANOOGA\n", dat.loc[dat['city'] == 'CHATTANOOGA']['created_date'].max(), '\n', dat.loc[dat['city'] == 'CHATTANOOGA']['created_date'].min())

CHATTANOOGA
 12/31/2015 12:58:28 PM 
 01/01/2004 09:20:46 AM


In [85]:
print("NaN\n", dat.loc[dat['city'].isna()]['created_date'].max(), '\n', dat.loc[dat['city'].isna()]['created_date'].min())

NaN
 12/31/2017 11:54:50 AM 
 01/01/2017 04:15:14 PM


For two largest groups, ticket creation date ranges appear to overlap.

In [89]:
print("CHATTANOOGA block_level_longitude\n", "Blanks:", dat.loc[dat['city'] == 'CHATTANOOGA']['block_level_longitude'].isna().sum(), '\n', 
     "Total:", dat.loc[dat['city'] == 'CHATTANOOGA'].shape[0])

CHATTANOOGA block_level_longitude
 Blanks: 212049 
 Total: 1415024


In [90]:
print("NaN block_level_longitude\n", "Blanks:", dat.loc[dat['city'].isna()]['block_level_longitude'].isna().sum(), '\n', 
     "Total:", dat.loc[dat['city'].isna()].shape[0])

NaN block_level_longitude
 Blanks: 176443 
 Total: 179101


While comprising a smaller percentage, records missing a value in `block_level_longitude` could still belong 

In [31]:
dat.loc[dat['city'] == 'CHATTANOOGA']['description'].value_counts().to_csv('output/has_city_descriptions.csv')

In [29]:
dat.loc[dat['city'].isna()]['description'].value_counts()

311 - Sewer Billing Inquiry                                                                                                                                                                                                                                                                                                                              54820
NSC - Overgrowth (Not Vacant)                                                                                                                                                                                                                                                                                                                             1022
311 - Address Validation Problems                                                                                                                                                                                                                                                                         

In [5]:
dat['description'].value_counts().index[:100]

Index(['CWS - Brush Collection', 'CWS - Trash Flash/Bulk Item',
       '311 - Sewer Billing Inquiry', 'NSC - Overgrowth (Not Vacant)',
       'CWS - Garbage Cont RPR/RPL', 'CWS - Illegal Dumping', 'NSC - Litter',
       'CWS - Garbage Missed', 'CWS - Bagged Yard Waste',
       'CWS - Garbage Cont New', 'NSC - Housing', 'CWS - General Comm/Comp',
       'TO - Traffic Sign Maintenance - Routine',
       'NSC - Abandoned/Inoperable Vehicle', 'BLD - Miscellaneous Tasks',
       'CWS - Sway Car Request', 'CWS - Pothole',
       'SWM - Drainage/Erosion Problem', 'CWS - Recycling Enrollment',
       'CWS - Dead Animal Pickup', 'AS - Priority 2/Nuisance/At Large',
       'NSC - Overgrowth (Vacant Lot)', 'PAR - Park Reservations',
       'CWS - Storm Drainage Probs', 'PAR - Work Request',
       'CWS - Right of Way Mowing', 'CWS - Recycle Missed',
       'CWS - Tree Fallen/Branch', 'CWS - Garbage Gen Comm/Comp',
       'CWS - General Complaints', 'SEW - Sewer Backup (Home/Business)',
       'CW

In [6]:
descript_val_counts = dat['description'].value_counts()
descript_val_counts.head(10)

CWS - Brush Collection           397463
CWS - Trash Flash/Bulk Item      262081
311 - Sewer Billing Inquiry       54822
NSC - Overgrowth (Not Vacant)     49741
CWS - Garbage Cont RPR/RPL        49382
CWS - Illegal Dumping             47293
NSC - Litter                      42262
CWS - Garbage Missed              40855
CWS - Bagged Yard Waste           38958
CWS - Garbage Cont New            25808
Name: description, dtype: int64

In [ ]:
descript_val_counts.head()

In [13]:
# contains_code = descript_val_counts['description'].str.contains('^\w{2,3} - ')
contains_code = dat['description'].str.contains('^\w{2,3} - ')
# descript_codes = descript_val_counts[contains_code]

In [ ]:
descript_codes.to_csv('output\descript_codes.csv')

In [17]:
contains_code[~contains_code.isna()]

0           True
1           True
2           True
3           True
4           True
5           True
6           True
7           True
8           True
9           True
10          True
11          True
12          True
13          True
14          True
15          True
16          True
17          True
18          True
19          True
20          True
21          True
22          True
23          True
24          True
25          True
26          True
27          True
28          True
29          True
           ...  
1693764    False
1693769    False
1693773    False
1693777    False
1693778    False
1693779    False
1693780    False
1693782    False
1693783    False
1693785    False
1693791    False
1693792    False
1693793    False
1693800    False
1693801    False
1693802    False
1693803    False
1693805    False
1693827    False
1693830    False
1693838    False
1693842    False
1693843    False
1693848    False
1693849    False
1693850    False
1693853    False
1693861    Fal